In [1]:
# импортируем необходимые библиотеки, функцию train_test_split()
# и классы SimpleImputer, StandardScaler, OneHotEncoder, 
# ColumnTransformer, LogisticRegression, Pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head(5)

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8293 entries, 0 to 8292
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer Lifetime Value        8289 non-null   float64
 1   Coverage                       8288 non-null   object 
 2   Education                      8290 non-null   object 
 3   EmploymentStatus               8288 non-null   object 
 4   Gender                         8289 non-null   object 
 5   Income                         8291 non-null   float64
 6   Monthly Premium Auto           8282 non-null   float64
 7   Months Since Last Claim        8288 non-null   float64
 8   Months Since Policy Inception  8285 non-null   float64
 9   Number of Open Complaints      8287 non-null   float64
 10  Number of Policies             8288 non-null   float64
 11  Response                       8293 non-null   object 
dtypes: float64(7), object(5)
memory usage: 777.6+ KB

In [4]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [5]:
# создаем списки категориальных 
# и количественных столбцов
cat_columns = X_train.select_dtypes(
    include='object').columns.tolist()
num_columns = X_train.select_dtypes(
    exclude='object').columns.tolist()

In [6]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [7]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

In [8]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)
# смотрим порядок столбцов, заданный теперь ColumnTransformer
transformer

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Customer Lifetime Value', 'Income',
                                  'Monthly Premium Auto',
                                  'Months Since Last Claim',
                                  'Months Since Policy Inception',
                                  'Number of Open Complaints',
                                  'Number of Policies']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                

In [9]:
# задаем итоговый конвейер
ml_pipe = Pipeline(
    [('transform', transformer), 
     ('logreg', LogisticRegression(solver='lbfgs', 
                                   max_iter=200))])

In [10]:
# обучаем итоговый конвейер
ml_pipe.fit(X_train, y_train)
# оцениваем качество конвейера на обучающих данных
print("Правильность на обучающей выборке: {:.3f}".format(
    ml_pipe.score(X_train, y_train)))
# оцениваем качество конвейера на тестовых данных
print("Правильность на тестовой выборке: {:.3f}".format(
    ml_pipe.score(X_test, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.899


In [11]:
# извлекаем дамми-переменные, созданные классом OneHotEncoder
cat = ml_pipe.named_steps['transform'].named_transformers_['cat']
onehot_columns = list(cat.named_steps['ohe'].get_feature_names_out(
    input_features=cat_columns))
onehot_columns

['Coverage_Basic',
 'Coverage_Extended',
 'Coverage_Premium',
 'Education_Bachelor',
 'Education_College',
 'Education_Doctor',
 'Education_High School or Below',
 'Education_Master',
 'EmploymentStatus_Disabled',
 'EmploymentStatus_Employed',
 'EmploymentStatus_Medical Leave',
 'EmploymentStatus_Retired',
 'EmploymentStatus_Unemployed',
 'Gender_F',
 'Gender_M']

In [12]:
# еще можно применить такой стиль
cat = ml_pipe.named_steps.transform.named_transformers_.cat
onehot_columns = list(cat.named_steps.ohe.get_feature_names_out(
    input_features=cat_columns))
onehot_columns

['Coverage_Basic',
 'Coverage_Extended',
 'Coverage_Premium',
 'Education_Bachelor',
 'Education_College',
 'Education_Doctor',
 'Education_High School or Below',
 'Education_Master',
 'EmploymentStatus_Disabled',
 'EmploymentStatus_Employed',
 'EmploymentStatus_Medical Leave',
 'EmploymentStatus_Retired',
 'EmploymentStatus_Unemployed',
 'Gender_F',
 'Gender_M']

In [13]:
# добавляем в конец списка количественных переменных 
# дамми-переменные, созданные OneHotEncoder, т.е. 
# сохраняем тот же порядок столбцов, что задал 
# ColumnTransformer
all_columns_lst = num_columns + onehot_columns
all_columns_lst

['Customer Lifetime Value',
 'Income',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies',
 'Coverage_Basic',
 'Coverage_Extended',
 'Coverage_Premium',
 'Education_Bachelor',
 'Education_College',
 'Education_Doctor',
 'Education_High School or Below',
 'Education_Master',
 'EmploymentStatus_Disabled',
 'EmploymentStatus_Employed',
 'EmploymentStatus_Medical Leave',
 'EmploymentStatus_Retired',
 'EmploymentStatus_Unemployed',
 'Gender_F',
 'Gender_M']

In [14]:
# извлекаем константу
intercept = np.round(ml_pipe.named_steps['logreg'].intercept_[0], 3)
intercept

-1.697

In [15]:
# извлекаем коэффициенты
coef = np.round(ml_pipe.named_steps['logreg'].coef_, 3)
coef

array([[ 7.000e-03,  2.300e-02,  1.240e-01, -4.700e-02, -2.900e-02,
        -4.000e-02, -5.900e-02, -3.600e-02,  1.290e-01, -9.400e-02,
        -1.640e-01, -2.100e-02,  4.120e-01, -1.710e-01, -5.700e-02,
         1.000e-03, -5.240e-01, -1.000e-01,  1.648e+00, -1.026e+00,
        -7.000e-03,  6.000e-03]])

In [16]:
# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], all_columns_lst):
    print(feature, c)

Константа: -1.697
Регрессионные коэффициенты:
Customer Lifetime Value 0.007
Income 0.023
Monthly Premium Auto 0.124
Months Since Last Claim -0.047
Months Since Policy Inception -0.029
Number of Open Complaints -0.04
Number of Policies -0.059
Coverage_Basic -0.036
Coverage_Extended 0.129
Coverage_Premium -0.094
Education_Bachelor -0.164
Education_College -0.021
Education_Doctor 0.412
Education_High School or Below -0.171
Education_Master -0.057
EmploymentStatus_Disabled 0.001
EmploymentStatus_Employed -0.524
EmploymentStatus_Medical Leave -0.1
EmploymentStatus_Retired 1.648
EmploymentStatus_Unemployed -1.026
Gender_F -0.007
Gender_M 0.006


In [17]:
# импортируем функции make_pipeline() и make_column_transformer(), 
# класс make_column_selector 
from sklearn.compose import (make_column_transformer, 
                             make_column_selector)
from sklearn.pipeline import make_pipeline

In [18]:
# автоматически создаем экземпляр класса ColumnTransformer,
# при этом автоматически создав трансформеры и списки
# признаков
column_transformer_quick = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy='median'), 
                   StandardScaler()),
     make_column_selector(dtype_include=np.number)),  
    (make_pipeline(SimpleImputer(strategy='most_frequent'), 
                   OneHotEncoder()),
     make_column_selector(dtype_include=object)))
column_transformer_quick

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f8670a61400>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f8670a61550>)])

In [19]:
# автоматически создаем итоговый конвейер
pipe_quick = make_pipeline(
    column_transformer_quick, 
    LogisticRegression(solver='lbfgs', max_iter=200))
pipe_quick 

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f8670a61400>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                              

In [20]:
# обучаем итоговый конвейер
pipe_quick.fit(X_train, y_train)
# оцениваем качество конвейера на обучающих данных
print("Правильность на обучающей выборке: {:.3f}".format(
    pipe_quick.score(X_train, y_train)))
# оцениваем качество конвейера на тестовых данных
print("Правильность на тестовой выборке: {:.3f}".format(
    pipe_quick.score(X_test, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.899


In [21]:
# извлекаем дамми-переменные, 
# созданные классом OneHotEncoder
cat = pipe_quick.named_steps['columntransformer']\
                .named_transformers_['pipeline-2']
onehot_columns = list(cat.named_steps['onehotencoder'].get_feature_names_out(
    input_features=cat_columns))
onehot_columns

['Coverage_Basic',
 'Coverage_Extended',
 'Coverage_Premium',
 'Education_Bachelor',
 'Education_College',
 'Education_Doctor',
 'Education_High School or Below',
 'Education_Master',
 'EmploymentStatus_Disabled',
 'EmploymentStatus_Employed',
 'EmploymentStatus_Medical Leave',
 'EmploymentStatus_Retired',
 'EmploymentStatus_Unemployed',
 'Gender_F',
 'Gender_M']

In [22]:
# добавляем в конец списка количественных переменных 
# дамми-переменные, созданные OneHotEncoder, т.е. 
# сохраняем тот же порядок столбцов, что задал 
# ColumnTransformer
all_columns_lst = num_columns + onehot_columns
all_columns_lst

['Customer Lifetime Value',
 'Income',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies',
 'Coverage_Basic',
 'Coverage_Extended',
 'Coverage_Premium',
 'Education_Bachelor',
 'Education_College',
 'Education_Doctor',
 'Education_High School or Below',
 'Education_Master',
 'EmploymentStatus_Disabled',
 'EmploymentStatus_Employed',
 'EmploymentStatus_Medical Leave',
 'EmploymentStatus_Retired',
 'EmploymentStatus_Unemployed',
 'Gender_F',
 'Gender_M']

In [23]:
# извлекаем константу
logreg_step = pipe_quick.named_steps['logisticregression']
intercept = np.round(logreg_step.intercept_[0], 3)
# извлекаем коэффициенты
coef = np.round(logreg_step.coef_, 3)

In [24]:
# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], all_columns_lst):
    print(feature, c)

Константа: -1.697
Регрессионные коэффициенты:
Customer Lifetime Value 0.007
Income 0.023
Monthly Premium Auto 0.124
Months Since Last Claim -0.047
Months Since Policy Inception -0.029
Number of Open Complaints -0.04
Number of Policies -0.059
Coverage_Basic -0.036
Coverage_Extended 0.129
Coverage_Premium -0.094
Education_Bachelor -0.164
Education_College -0.021
Education_Doctor 0.412
Education_High School or Below -0.171
Education_Master -0.057
EmploymentStatus_Disabled 0.001
EmploymentStatus_Employed -0.524
EmploymentStatus_Medical Leave -0.1
EmploymentStatus_Retired 1.648
EmploymentStatus_Unemployed -1.026
Gender_F -0.007
Gender_M 0.006


In [25]:
# создаем массив булевых значений
categorical = X_train.dtypes == object
categorical

Customer Lifetime Value          False
Coverage                          True
Education                         True
EmploymentStatus                  True
Gender                            True
Income                           False
Monthly Premium Auto             False
Months Since Last Claim          False
Months Since Policy Inception    False
Number of Open Complaints        False
Number of Policies               False
dtype: bool

In [26]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, ~categorical),
                ('cat', cat_pipe, categorical)]

In [27]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
ml_pipe = Pipeline(
    [('transform', transformer), 
     ('logreg', LogisticRegression(solver='lbfgs', 
                                   max_iter=200))])
# обучаем итоговый конвейер
ml_pipe.fit(X_train, y_train)
# оцениваем качество конвейера на обучающих данных
print("Правильность на обучающей выборке: {:.3f}".format(
    ml_pipe.score(X_train, y_train)))
# оцениваем качество конвейера на тестовых данных
print("Правильность на тестовой выборке: {:.3f}".format(
    ml_pipe.score(X_test, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.899


In [28]:
# извлекаем дамми-переменные, созданные классом OneHotEncoder
cat = ml_pipe.named_steps['transform'].named_transformers_['cat']
onehot_cols = list(cat.named_steps['ohe'].get_feature_names_out(
    input_features=X_train.loc[:, categorical].columns))
num_cols = X_train.loc[:, ~categorical].columns.tolist()

# добавляем в конец списка количественных переменных 
# дамми-переменные, созданные OneHotEncoder, т.е. 
# сохраняем тот же порядок столбцов, что задал 
# ColumnTransformer
all_cols_lst = num_cols + onehot_cols

# извлекаем константу
lr_step = ml_pipe.named_steps['logreg']
intercept = np.round(lr_step.intercept_[0], 3)
# извлекаем коэффициенты
coef = np.round(lr_step.coef_, 3)

# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], all_cols_lst):
    print(feature, c)

Константа: -1.697
Регрессионные коэффициенты:
Customer Lifetime Value 0.007
Income 0.023
Monthly Premium Auto 0.124
Months Since Last Claim -0.047
Months Since Policy Inception -0.029
Number of Open Complaints -0.04
Number of Policies -0.059
Coverage_Basic -0.036
Coverage_Extended 0.129
Coverage_Premium -0.094
Education_Bachelor -0.164
Education_College -0.021
Education_Doctor 0.412
Education_High School or Below -0.171
Education_Master -0.057
EmploymentStatus_Disabled 0.001
EmploymentStatus_Employed -0.524
EmploymentStatus_Medical Leave -0.1
EmploymentStatus_Retired 1.648
EmploymentStatus_Unemployed -1.026
Gender_F -0.007
Gender_M 0.006
